In [1]:
import os.path

import torch
import matplotlib.pyplot as plt
import torch.nn as nn


In [32]:
# from clint.textui import progress
import requests
from  tqdm import tqdm
import zipfile
import os
def get_data_from_dataset(dataset:str):
    horse2zebra_url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip'
    res = requests.get(horse2zebra_url,stream=True)
    total_length = int(res.headers.get('content-length'))
    chunk_size = 1024*1024
    zip_path='./data'
    file_name = "%s.zip" % dataset
    is_download = True
    is_extract = True
    if os.path.exists(file_name):
        now_size = os.path.getsize(file_name)
        is_download=False
        if now_size!=total_length:
            is_download=True
        if is_download:
            print("now downloading ")
        else:
            print("had downloaded")
    if is_download:
        with open(file_name, "wb") as zip_file:
            for chunk in tqdm(iterable=res.iter_content(chunk_size=chunk_size),total=total_length/chunk_size,desc=dataset,unit='MB'):
                # print("write i")
                zip_file.write(chunk)
    if not os.path.exists(zip_path):
        os.makedirs(zip_path)
    else:
        print('path exist')
    full_name = zip_path+'/'+dataset
    if os.path.exists(full_name):
        # print("file ex")
        is_extract=False
    # print(full_name)
    if is_extract:
        zf = zipfile.ZipFile(file_name,mode='r')
        for f_n in zf.namelist():
            zf.extract(f_n,path=zip_path)
        zf.close()
        print("extract finished ")



had downloaded
path exist
extract finished 
